# TRANSFER LEARNING FOR ANXIETY
In this notebook, I'm going to train a model for PHQ-8 with all the possible parameters. After, I'm going to save the model and use it as a pretrained model over GAD-7 information. 

Import all the facilities

In [1]:
from sklearn.model_selection import KFold
from data_Loader import dataLoaderPickle, justDatasetLoaderPickle
from networks_v2 import AttentionModel
from optimizer import train2, test2, optimizerNet, test, train
from optimizer import IterMeter

import torch
import torch.nn as nn
from comet_ml import Experiment

import numpy as np

from torch.utils.data import DataLoader, random_split, SubsetRandomSampler

c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading datasets

In [2]:
train_dataDep, test_dataDep = dataLoaderPickle('BALANCED_DATASET_ENG_SCRIPTED_FULL')
dataset_anx = justDatasetLoaderPickle('Scripted_Eng_GAD7')

Training size: 11916
Testing size: 2979


In [ ]:
tar = []
import matplotlib.pyplot as plt
for x, y in dataset_anx:
    tar.append(y)

n, bins, patches = plt.hist(x=tar,bins='auto',color='#0504aa',alpha=0.7, rwidth=0.85)
plt.grid(axis='y',alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Is my data balanced?')
plt.text(23,45,r'$\mu15, b=3$')
maxfreq = n.max()
plt.ylim(ymax=np.ceil(maxfreq/10)*10 if maxfreq % 10 else maxfreq + 10)
plt.show()

## Training and saving the PHQ-8 Model

In [10]:
#Constants
learning_Rate = 0.0005
batch_size = 128
epochs = 200
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 5,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [4]:
#Building and training Model
model = AttentionModel(
    hparams['n_cnn_layers'],
    hparams['rnn_dim'],
    hparams['h_rnn_layers'],
    hparams['n_rnn_layers'],
    hparams['n_class'],
    hparams['stride'],
    hparams['dropout']
).to(device).float()

print(model)
print('Number Model Parameters', sum([param.nelement() for param in model.parameters()]))

#Optimizing Model
optimizer, scheduler = optimizerNet(model, hparams)
#criterion = nn.NLLLoss().to(device)
weights = [0.95, 0.85, 1.0, 0.9, 1.0]
class_weights = torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
criterion2 = nn.MSELoss()
iter_meter = IterMeter()
#for epoch in range(1, epochs + 1):
train(model, device, train_dataDep, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
test(model, device, test_dataDep, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

AttentionModel(
  (dense): Conv1d(500, 32, kernel_size=(1,), stride=(1,))
  (cnn): Sequential(
    (0): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (2): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_runn

In [9]:
path = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\model_PHQ8_Scripted_Eng.pt"
output = open(path, mode="wb")
torch.save(model,output)
output.close()

## Training GAD-7 with PHQ-8 Model

First, let's build a new class, which will be the pretrained model changing the last FC Layer with 5 classes for the 4 classes one at GAD-7.

In [3]:
class AnxietyFromDepression(nn.Module):
    def __init__(self, pretrained_model):
        super(AnxietyFromDepression, self).__init__()
        self.pretrained = pretrained_model
        self.new_layer = nn.Linear(5,4)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.pretrained(x)
        x = self.new_layer(x)
        x = self.activation(x)
        return x

In [7]:
filename = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\model_PHQ8_Scripted_Eng.pt"
infile = open(filename, 'rb')
model_pretrained = torch.load(infile)
infile.close()

In [11]:
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

model_TF = AnxietyFromDepression(model_pretrained)

#Optimizing Model
optimizer, scheduler = optimizerNet(model_TF, hparams)
weights = [1.0,1.0,1.0,0.85]
class_weights = torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
iter_meter = IterMeter()

In [13]:
epochs = 50

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_anx)))):
    print('Fold {}'.format(fold+1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loaderAnx = DataLoader(dataset_anx, batch_size=batch_size, sampler=train_sampler)
    test_loaderAnx = DataLoader(dataset_anx, batch_size=batch_size, sampler=test_sampler)

    model_TF = AnxietyFromDepression(model_pretrained)

    optimizer, scheduler = optimizerNet(model_TF, hparams)

    train2(model_TF, device, train_loaderAnx, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
    test2(model_TF, device, test_loaderAnx, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

Fold 1
Train Epoch: 0 	Loss: 1.4520	Accuracy: 0.1706
Train Epoch: 1 	Loss: 1.3913	Accuracy: 0.1926
Train Epoch: 2 	Loss: 1.3663	Accuracy: 0.2036
Train Epoch: 3 	Loss: 1.3510	Accuracy: 0.2126
Train Epoch: 4 	Loss: 1.3399	Accuracy: 0.2202
Train Epoch: 5 	Loss: 1.3311	Accuracy: 0.2273
Train Epoch: 6 	Loss: 1.3233	Accuracy: 0.2358
Train Epoch: 7 	Loss: 1.3165	Accuracy: 0.2445
Train Epoch: 8 	Loss: 1.3102	Accuracy: 0.2538
Train Epoch: 9 	Loss: 1.3044	Accuracy: 0.2625
Train Epoch: 10 	Loss: 1.2987	Accuracy: 0.2705
Train Epoch: 11 	Loss: 1.2932	Accuracy: 0.2782
Train Epoch: 12 	Loss: 1.2879	Accuracy: 0.2899
Train Epoch: 13 	Loss: 1.2827	Accuracy: 0.3009
Train Epoch: 14 	Loss: 1.2777	Accuracy: 0.3109
Train Epoch: 15 	Loss: 1.2729	Accuracy: 0.3199
Train Epoch: 16 	Loss: 1.2682	Accuracy: 0.3281
Train Epoch: 17 	Loss: 1.2635	Accuracy: 0.3353
Train Epoch: 18 	Loss: 1.2590	Accuracy: 0.3422
Train Epoch: 19 	Loss: 1.2545	Accuracy: 0.3484
Train Epoch: 20 	Loss: 1.2502	Accuracy: 0.3539
Train Epoch: 21 

Full training with 150 epochs to compare with the non pre-trained model. From this result we will be able to adjust and confirm possibilities with Multi-Task Learning.

In [15]:
train_loaderAnx, test_loaderAnx = dataLoaderPickle('Scripted_Eng_GAD7')
epochs = 150
model_TF = AnxietyFromDepression(model_pretrained)
optimizer, scheduler = optimizerNet(model_TF, hparams)
train2(model_TF, device, train_loaderAnx, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
test2(model_TF, device, test_loaderAnx, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

Training size: 10772
Testing size: 2693
Train Epoch: 0 	Loss: 1.3620	Accuracy: 0.3665
Train Epoch: 1 	Loss: 1.3340	Accuracy: 0.3903
Train Epoch: 2 	Loss: 1.3137	Accuracy: 0.4069
Train Epoch: 3 	Loss: 1.2996	Accuracy: 0.4204
Train Epoch: 4 	Loss: 1.2886	Accuracy: 0.4299
Train Epoch: 5 	Loss: 1.2804	Accuracy: 0.4391
Train Epoch: 6 	Loss: 1.2728	Accuracy: 0.4459
Train Epoch: 7 	Loss: 1.2660	Accuracy: 0.4522
Train Epoch: 8 	Loss: 1.2598	Accuracy: 0.4575
Train Epoch: 9 	Loss: 1.2541	Accuracy: 0.4615
Train Epoch: 10 	Loss: 1.2487	Accuracy: 0.4655
Train Epoch: 11 	Loss: 1.2433	Accuracy: 0.4684
Train Epoch: 12 	Loss: 1.2383	Accuracy: 0.4708
Train Epoch: 13 	Loss: 1.2335	Accuracy: 0.4725
Train Epoch: 14 	Loss: 1.2289	Accuracy: 0.4742
Train Epoch: 15 	Loss: 1.2244	Accuracy: 0.4758
Train Epoch: 16 	Loss: 1.2202	Accuracy: 0.4776
Train Epoch: 17 	Loss: 1.2160	Accuracy: 0.4791
Train Epoch: 18 	Loss: 1.2121	Accuracy: 0.4801
Train Epoch: 19 	Loss: 1.2083	Accuracy: 0.4811
Train Epoch: 20 	Loss: 1.2044	